# 🌊 Exploring Copernicus Marine Data in Google Colab

**An interactive notebook for retrieving and exploring Copernicus Marine datasets, created by Náttúrufræðistofnun.**

---

### Overview
This notebook connects to **Copernicus Marine Service** to retrieve and list all available datasets.
Using Python, it fetches dataset descriptions automatically and generates markdown documentation.

The script logs in Copernicus Marine using credentials and queries Copernicus Marine API for metadata.

Find a full list of available products here: https://data.marine.copernicus.eu/products

### How to Use
1. Open this notebook in Google Colab or another cloud platform.
2. Register a Copernicus Marine account here: https://data.marine.copernicus.eu/register
3. Insert valid credentials in section Login to access the data.
4. Run the provided code cells in sequence (do not skip cells) to explore dataset descriptions and available services, download, process and display them.

| Try the code via free cloud platforms: | [![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lmi/Copernicus/blob/master/Marine/Overview/ConnectMarine.ipynb)

---

### Attribution
- **License**: This notebook is published under **CC BY 4.0**.
- **Citation**: "*Credits: Náttúrufræðistofnun*"
- **Author**: Marco Pizzolato
- **Data Sources**: [Copernicus Marine Service](https://marine.copernicus.eu/)
- **Created in**: Python

---

**This notebook provides an automated way to explore and document Copernicus Marine datasets, making ocean data more accessible.**


## Install and Import

In [ ]:
!pip install -q copernicusmarine pandas tabulate Pillow mdutils leafmap rioxarray xarray netCDF4 rasterio pyproj localtileserver

In [ ]:
import os
import requests
from PIL import Image
from mdutils.mdutils import MdUtils
from mdutils.tools import Html

## Login to Copernicus Marine

Register a Copernicus Marine account here: https://data.marine.copernicus.eu/register

In [ ]:
import copernicusmarine as cm
import getpass

username = input("Enter your Copernicus Marine username: ")
password = getpass.getpass("Enter your Copernicus Marine password: ")

cm.login(username=username, password=password)

## Fetch Catalog

In [ ]:
catalog = cm.describe()

In [ ]:
# Inspect the structure of the first product and its datasets
first_product = catalog.products[0]
print(first_product.model_dump())
print(first_product.datasets)

## Save catalog as Markdown File

In [ ]:
# Initialize the Markdown file
md_file = MdUtils(file_name='Copernicus_Marine_Products_and_Datasets')

# Iterate through products
for product in catalog.products:
    product_title = product.title
    product_id = product.product_id
    description = product.description
    thumbnail_url = product.thumbnail_url

        # Add thumbnail image if available
    if thumbnail_url:
        # Resize the image to 100x100 pixels using HTML
        img_tag = Html.image(path=thumbnail_url, size='300x200')
        md_file.new_paragraph(img_tag)

    # Add an empty line after the image
    md_file.new_paragraph('\n')

    # Add a header for the product
    md_file.new_header(level=1, title=product_title)

    # Add product ID
    md_file.new_paragraph(f'**Product ID:** {product_id}')

    # Add description
    md_file.new_paragraph(f'**Description:** {description}')

    # Add an empty line after the image
    md_file.new_paragraph('\n')

In [ ]:
# Create the Markdown file
md_file.create_md_file()

## Display the created overview

In [ ]:
from IPython.display import display, Markdown

In [ ]:
# Display the generated markdown file at the end of the notebook
n_lines = 100  # how many lines to display
with open("Copernicus_Marine_Products_and_Datasets.md", "r") as file:
    content = "".join([next(file) for _ in range(n_lines)])
display(Markdown(content + "\n\n... *truncated*"))

## Download data

In [ ]:
import datetime as dt
# Product example: ARCTIC_ANALYSISFORECAST_BGC_002_004
DATASET_ID = "cmems_mod_arc_bgc_anfc_ecosmo_P1D-m"  #  ⟵ from product page (Select one of you choice)

# Small AOI covering Iceland and nearby Arctic (adjust as you wish)
bbox = dict(
    minimum_longitude=-40.0, maximum_longitude=40.0,
    minimum_latitude=60.0,  maximum_latitude=85.0
)

# Use "latest" day that is surely produced (yesterday is safe for NRT)
end = dt.datetime.utcnow().date()
start = end - dt.timedelta(days=1)
time_sel = dict(
    start_datetime=start.isoformat(),
    end_datetime=end.isoformat()
)

# Name of the file we download
out_nc = "arctic_bgc_chl.nc"

In [ ]:
# We'll ask for "chl*" but stay robust: if variable name differs, we'll detect it later from the file itself.
# Request only surface layer (0 m). Many CMEMS BGC datasets use "depth" in meters.
cm.subset(
    dataset_id=DATASET_ID,
    variables=None,                      # keep None here; we’ll auto-pick "chl" variable after download
    **bbox,
    minimum_depth=0, maximum_depth=0,    # surface
    **time_sel,
    output_filename=out_nc
)

## Convert the data to TIFF (to plot on map)

In [ ]:
import xarray as xr
import rioxarray as rxr

ds = xr.open_dataset(out_nc, decode_times=True)

# ---- Find a chlorophyll-like variable (robust to name variations)
candidates = [v for v in ds.data_vars if ("chl" in v.lower()) or ("chlor" in v.lower())]
if not candidates:
    raise ValueError(f"No chlorophyll-like variable found in {list(ds.data_vars)}")
var = candidates[0]
da = ds[var]

# ---- Select a single time slice (nearest to end date)
if "time" in da.dims:
    da = da.sel(time=da["time"].max())

# ---- Some CMEMS grids are polar stereographic with x/y & grid_mapping.
# Try to attach CRS using the CF grid mapping, otherwise assume lat/lon.
def write_geotiff_from_da(da, tif_path="arctic_bgc_chl.tif"):
    # If we have lat/lon coords, reproject on-the-fly by assigning EPSG:4326
    if set(["lat", "latitude"]).intersection(da.coords) and set(["lon", "longitude"]).intersection(da.coords):
        # Normalize coordinate names
        lat_name = "lat" if "lat" in da.coords else "latitude"
        lon_name = "lon" if "lon" in da.coords else "longitude"
        da = da.rename({lat_name: "lat", lon_name: "lon"})
        # rioxarray expects y,x order named differently; use .rio.set_spatial_dims
        da = da.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=False)
        da = da.rio.write_crs(4326, inplace=False)
        da.rio.to_raster(tif_path)
        return tif_path

    # Else assume projected x/y with CF grid_mapping
    # rioxarray can parse CRS from the grid mapping variable when present.
    # Find 'x','y' or 'rlon','rlat' etc., normalize to x/y.
    spatial_dims = [d for d in da.dims if d.lower() in ("x","y","rlon","rlat","cols","rows")]
    if len(spatial_dims) >= 2:
        # Try to infer x/y order
        # Prefer something like ('x','y'); else sort by "x-like" first
        order = []
        for cand in ("x","rlon","cols"):
            if cand in da.dims: order.append(cand)
        for cand in ("y","rlat","rows"):
            if cand in da.dims: order.append(cand)
        # Fallback: keep existing
        if len(order) != 2: order = da.dims[:2]
        da = da.rename({order[0]: "x", order[1]: "y"})
        # Attach CRS from grid mapping if available
        gm = da.rio.grid_mapping
        if gm is None:
            # As a last resort, try common polar stereographic for ARC MFC
            # (Best effort; proper CRS comes from CF metadata.)
            da = da.rio.write_crs("EPSG:3857", inplace=False)  # Arctic Polar Stereographic
        # Ensure spatial dims are set
        da = da.rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=False)
        da.rio.to_raster(tif_path)
        return tif_path

    raise RuntimeError("Could not determine geospatial coordinates/CRS to write GeoTIFF.")

tif_path = write_geotiff_from_da(da, "arctic_bgc_chl.tif")
tif_path

## Display the data on a map

In [ ]:
import leafmap

# Initialize map
m = leafmap.Map(center=[70, -20], zoom=3)

# Add background layer
m.add_basemap("Esri.OceanBasemap")

# Add our downloaded raster
m.add_raster(
    tif_path,
    layer_name="Chlorophyll-a (surface)",
    zoom_to_layer=True,
    colormap="viridis",
    vmin=0, # Min value for color scale
    vmax=10 # Max value for color scale
)

# Display map
m

## Display data directly from the web with no download

In [ ]:
import leafmap

# The layer identifier you can find them all here: https://data.marine.copernicus.eu/products
LAYER_ID = ("ARCTIC_ANALYSISFORECAST_BGC_002_004/"
            "cmems_mod_arc_bgc_anfc_ecosmo_P1D-m_202105/chl")

# Pick any of the listed times;
ISO_TIME = "2025-07-01T00:00:00Z"

# Optional: elevation (depth, in meters). The default in the XML is ~ -0.494 m (surface layer in z*)
ELEVATION = "-0.4940253794193268"   # or omit the &ELEVATION param to use the default

# Choose a TileMatrixSet that matches your basemap (Web Mercator or WGS84)
TMS = "EPSG:3857"

# Server-side styling. If viridis isn’t available on this layer, switch to 'cmap:matter' or use 'logScale' variant.
STYLE = "cmap:viridis"      # fallback: "cmap:matter" or "cmap:matter,logScale"

# Make URL
WMTS_URL = (
    "https://wmts.marine.copernicus.eu/teroWmts"
    "?SERVICE=WMTS&REQUEST=GetTile&VERSION=1.0.0"
    f"&LAYER={LAYER_ID}"
    "&FORMAT=image/png"
    f"&TILEMATRIXSET={TMS}"
    "&TILEMATRIX={z}&TILEROW={y}&TILECOL={x}"
    f"&time={ISO_TIME}"
    f"&STYLE={STYLE}"
    f"&ELEVATION={ELEVATION}"
)

# Prepare map
m = leafmap.Map(center=[30, 0], zoom=2)

# Add our WMTS layer
m.add_tile_layer(
    url=WMTS_URL,
    name="CMEMS Zooplankton (monthly)",
    attribution="Copernicus Marine"
)
m

The difference is that on the online data served as Web Map Tile Server (WMTS) it is not possible to process the data, while on the downloaded data it is possibe to combine it with other data or extract the single pixel values.